In [3]:
!pip install selenium bs4

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.1/9.6 MB 1.5 MB/s eta 0:00:07
    --------------------------------------- 0.2/9.6 MB 1.8 MB/s eta 0:00:06
   - -------------------------------------- 0.4/9.6 MB 2.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/9.6 MB 2.5 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/9.6 MB 3.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.6 MB 5.2 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.6 MB 8.1 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 10.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.3/9.6 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------  9.6/9.6 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------  9.6/9.6 MB 19.1 MB/s eta 0:00:01
   -------------

In [21]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

import os

from time import sleep

In [137]:
# Функция для ожидания кликабельного элемента
def wait_click_func(*args):
    wait_click = WebDriverWait(driver, timeout=10)
    wait_click.until(EC.element_to_be_clickable(args)).click()

if __name__ == '__main__':

    url = 'https://apps.sentinel-hub.com/eo-browser'
    data_path = os.path.abspath(os.getcwd()) + "\\data"

    sec_to_wait = 100

    
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", data_path)
    #options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
    
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    driver.maximize_window()

    
    # Нажатие кнопки использования без авторизации
    wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
    
    # Нажатие кнопки для пропуска туториала
    wait_click_func(By.XPATH, "//button[@class='tutorial-button tutorial-closebutton']")


    # Ввод местоположения откуда хотим снимки
    place = "Moscow"
    driver.find_element(By.XPATH, '//input[@placeholder="Go to Place"]').send_keys(place)
    wait_click_func(By.XPATH, "//div[@class='search-item']")
    
    # Нажатие кнопки поиска снимков
    wait_click_func(By.XPATH, "//div[@class='search-btn-wrapper']")
    #print(driver.find_element(By.XPATH, "//div[@class='tools-container']").get_attribute('innerHTML'))
    #driver.find_element(By.XPATH, "//li[@id='SearchTabButton']").click()
    
    wait = WebDriverWait(driver, timeout=sec_to_wait)
    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='showing-n-results']")))


    for i in range(50):
        # Нажатие на первый снимок
        sleep(2)
        wait_click_func(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//img[@class='preview-image']")
        
        # Найти все снимки
        # satellite_pics_list = driver.find_element(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//div[@class='results-list']").find_elements(By.CLASS_NAME,'result-item')
        # sleep(1)
        # satellite_pics_list[i].find_element(By.XPATH,"//div[@class='img-btn']").click()
        
        # Если есть несколько вариантов, нажимаем на первый
        sleep(1)
        if driver.find_elements(By.CLASS_NAME, 'rodal-dialog rodal-slideUp-enter'):
            wait_click_func(By.XPATH, "//div[@class='results-panel']//div[@class='result-item']//img[@class='preview-image']")
        
        # Кнопка центрирования
        wait_click_func(By.XPATH, "//i[@class='fa fa-crosshairs']")
    
        #Зум
        zoom = 13
        cur_url = driver.current_url
        # print(cur_url)
        # print(cur_url[cur_url.find('zoom='):cur_url[cur_url.find('zoom='):].find('&')+len(cur_url[:cur_url.find('zoom=')])])
        start_point_str = 'zoom='
        end_point_str = '&'
        cur_zoom = cur_url[cur_url.find(start_point_str):cur_url[cur_url.find(start_point_str):].find(end_point_str)+len(cur_url[:cur_url.find(start_point_str)])]
        cur_url = cur_url.replace(cur_zoom, f'zoom={zoom}')
        # print(cur_url)
        driver.get(cur_url)
        for _ in range(7):
            
            # Скачать True color
            
            # Нажатие кнопки использования без авторизации
            wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
    
            # Нажать true color
            wait_click_func(By.XPATH, "//div[@class='layer-datasource-picker']//img[@src='https://apps.sentinel-hub.com/eo-browser/previews/DEFAULT-THEME-bd86bc-1_TRUE_COLOR.png']")
            
            # Кнопки скачки
            wait_click_func(By.XPATH, "//img[@alt='download-icon']")
            wait_click_func(By.XPATH, "//i[@class='fa fa-download']")
        
            # Дождёмся когда скачается
            wait = WebDriverWait(driver, timeout=sec_to_wait)
            wait.until(EC.presence_of_element_located((By.XPATH, "//i[@class='fa fa-download']")))
        
            # Закрытие скачки
            wait_click_func(By.XPATH, "//span[@class='rodal-close']")
    
    
            # Скачать NDVI
            
            # Нажать NDVI
            wait_click_func(By.XPATH, "//div[@class='layer-datasource-picker']//img[@src='https://apps.sentinel-hub.com/eo-browser/previews/DEFAULT-THEME-bd86bc-3_NDVI.png']")
    
            # Кнопки скачки
            wait_click_func(By.XPATH, "//img[@alt='download-icon']")
            wait_click_func(By.XPATH, "//i[@class='fa fa-download']")
        
            # Дождёмся когда скачается
            wait = WebDriverWait(driver, timeout=sec_to_wait)
            wait.until(EC.presence_of_element_located((By.XPATH, "//i[@class='fa fa-download']")))
        
            # Закрытие скачки
            wait_click_func(By.XPATH, "//span[@class='rodal-close']")
    
        
            # Переход на след полигон
            cur_url = driver.current_url
            start_point_str = 'lat='
            end_point_str = '&'
            cur_lat = cur_url[cur_url.find(start_point_str):cur_url[cur_url.find(start_point_str):].find(end_point_str)+len(cur_url[:cur_url.find(start_point_str)])]
            cur_url = cur_url.replace(cur_lat, f'lat={float(cur_lat[4:])+0.7}')
            driver.get(cur_url)
            
        # Нажатие кнопки использования без авторизации
        wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
        
        # Переход на снимки
        wait_click_func(By.XPATH, "//li[@id='SearchTabButton']")


        # Переход дат
        wait = WebDriverWait(driver, timeout=sec_to_wait)
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='date-pickers-wrapper']")))
        dates_elems = driver.find_element(By.XPATH, "//div[@class='date-pickers-wrapper']").find_elements(By.CLASS_NAME,'date-picker-input-wrapper')
        # Переход первой даты
        dates_elems[0].click()
        wait_click_func(By.XPATH, "//div[@class='DayPicker-wrapper']//i[@class='fa fa-chevron-left']")
        # Переход второй даты
        dates_elems[1].click()
        wait_click_func(By.XPATH, "//div[@class='DayPicker-wrapper']//i[@class='fa fa-chevron-left']")

        # Нажатие кнопки поиска снимков
        wait_click_func(By.XPATH, "//div[@class='search-btn-wrapper']")


        

    
    #<i class="fa fa-download"></i>
    # page = driver.page_source
    # print(page)
    # wait_click_func(By.XPATH, "//path[@class='leaflet-interactive']")

In [ ]:
# Функция для ожидания кликабельного элемента
def wait_click_func(*args):
    wait_click = WebDriverWait(driver, timeout=10)
    wait_click.until(EC.element_to_be_clickable(args)).click()

if __name__ == '__main__':

    url = 'https://apps.sentinel-hub.com/eo-browser'
    data_path = os.path.abspath(os.getcwd()) + "\\data"

    
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", data_path)
    #options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
    
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    driver.maximize_window()

    
    # Нажатие кнопки использования без авторизации
    wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
    
    # Нажатие кнопки для пропуска туториала
    wait_click_func(By.XPATH, "//button[@class='tutorial-button tutorial-closebutton']")
    
    # Ввод местоположения откуда хотим снимки
    place = "Moscow"
    driver.find_element(By.XPATH, '//input[@placeholder="Go to Place"]').send_keys(place)
    wait_click_func(By.XPATH, "//div[@class='search-item']")
    
    # Нажатие кнопки поиска снимков
    wait_click_func(By.XPATH, "//div[@class='search-btn-wrapper']")
    #print(driver.find_element(By.XPATH, "//div[@class='tools-container']").get_attribute('innerHTML'))
    #driver.find_element(By.XPATH, "//li[@id='SearchTabButton']").click()
    
    # Нажатие на первый снимок
    sleep(2)
    wait_click_func(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//img[@class='preview-image']")
    
    # Найти все снимки #print(driver.find_element(By.XPATH, "//div[@id='SearchTab']//div[@class='results']//div[@class='results-list']").find_elements(By.CLASS_NAME,'result-item'))
    
    
    # Если есть несколько вариантов, нажимаем на первый
    sleep(1)
    if driver.find_elements(By.CLASS_NAME, 'rodal-dialog rodal-slideUp-enter'):
        wait_click_func(By.XPATH, "//div[@class='results-panel']//div[@class='result-item']//img[@class='preview-image']")

    # Кнопка центрирования
    wait_click_func(By.XPATH, "//i[@class='fa fa-crosshairs']")

    #Зум
    zoom = 13
    cur_url = driver.current_url
    # print(cur_url)
    # print(cur_url[cur_url.find('zoom='):cur_url[cur_url.find('zoom='):].find('&')+len(cur_url[:cur_url.find('zoom=')])])
    start_point_str = 'zoom='
    end_point_str = '&'
    cur_zoom = cur_url[cur_url.find(start_point_str):cur_url[cur_url.find(start_point_str):].find(end_point_str)+len(cur_url[:cur_url.find(start_point_str)])]
    cur_url = cur_url.replace(cur_zoom, f'zoom={zoom}')
    # print(cur_url)
    driver.get(cur_url)
    for i in range(3):
        
        # Скачать True color
        
        # Нажатие кнопки использования без авторизации
        wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")

        # Нажать true color
        wait_click_func(By.XPATH, "//div[@class='layer-datasource-picker']//img[@src='https://apps.sentinel-hub.com/eo-browser/previews/DEFAULT-THEME-bd86bc-1_TRUE_COLOR.png']")

        test = driver.execute_script("var performance = window.performance || window.mozPerformance || window.msPerformance || window.webkitPerformance || {}; var network = performance.getEntries() || {}; return network;")

        for item in test:
          print(item)
        
        # # Кнопки скачки
        # wait_click_func(By.XPATH, "//img[@alt='download-icon']")
        # wait_click_func(By.XPATH, "//i[@class='fa fa-download']")
    
        # # Дождёмся когда скачается
        # wait = WebDriverWait(driver, timeout=10)
        # wait.until(EC.presence_of_element_located((By.XPATH, "//i[@class='fa fa-download']")))
    
        # # Закрытие скачки
        # wait_click_func(By.XPATH, "//span[@class='rodal-close']")


        # # Скачать NDVI
        # # Нажать NDVI
        # wait_click_func(By.XPATH, "//div[@class='layer-datasource-picker']//img[@src='https://apps.sentinel-hub.com/eo-browser/previews/DEFAULT-THEME-bd86bc-3_NDVI.png']")

        # # Кнопки скачки
        # wait_click_func(By.XPATH, "//img[@alt='download-icon']")
        # wait_click_func(By.XPATH, "//i[@class='fa fa-download']")
    
        # # Дождёмся когда скачается
        # wait = WebDriverWait(driver, timeout=10)
   

In [79]:
import requests
headers = {'User-agent': 'Mozilla/5.0'}
response = requests.get('https://apps.sentinel-hub.com/eo-browser/?zoom=14&lat=57.54487&lng=35.39589&themeId=DEFAULT-THEME&visualizationUrl=U2FsdGVkX1%2FepgvG6E2WUC1LWyvzozgdjkA5aHcwtz4%2FebDwUxrBRALjOrL5NSiaoOk1%2FW87kt1n9TsdgnhdTam2mxn%2F%2FuoOrVZmT5TS38XrDAjkNPTPmc0J5eOcfLO4&datasetId=S2L2A&fromTime=sec_to_wait24-09-18T00%3A00%3A00.000Z&toTime=sec_to_wait24-09-18T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR&demSource3D=%22MAPZEN%22')
response.headers

{'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1404', 'Connection': 'keep-alive', 'Server': 'CloudFront', 'Date': 'Thu, 19 Sep 2024 10:56:06 GMT', 'Content-Encoding': 'gzip', 'Cache-Control': 'max-age=0', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 d51f8b07f1cd9f6dbf62bb0b0c961f1c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'FRA60-P10', 'X-Amz-Cf-Id': 'GODN3yvf402neMUlQqMMYEk1pMU961CUb_zsKYiunZGeVkIQc0nnwA=='}

In [85]:
url = 'https://apps.sentinel-hub.com/eo-browser/?zoom=14&lat=57.54487&lng=35.39589&themeId=DEFAULT-THEME&visualizationUrl=U2FsdGVkX1%2FepgvG6E2WUC1LWyvzozgdjkA5aHcwtz4%2FebDwUxrBRALjOrL5NSiaoOk1%2FW87kt1n9TsdgnhdTam2mxn%2F%2FuoOrVZmT5TS38XrDAjkNPTPmc0J5eOcfLO4&datasetId=S2L2A&fromTime=2024-09-18T00%3A00%3A00.000Z&toTime=2024-09-18T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR&demSource3D=%22MAPZEN%22'
data_path = os.path.abspath(os.getcwd()) + "\\data"


options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", data_path)
#options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

driver = webdriver.Firefox(options=options)
driver.get(url)
driver.maximize_window()
# Нажатие кнопки использования без авторизации
wait_click_func(By.XPATH, "//div[@class='eob-btn anon-btn']")
test = driver.execute_script("var performance = window.performance || window.mozPerformance || window.msPerformance || window.webkitPerformance || {}; var network = performance.getEntries() || {}; return network;")

for item in test:
  print(item)

{'connectEnd': 296, 'connectStart': 118, 'contentType': 'text/html', 'decodedBodySize': 3257, 'domComplete': 3727, 'domContentLoadedEventEnd': 2118, 'domContentLoadedEventStart': 2105, 'domInteractive': 555, 'domainLookupEnd': 118, 'domainLookupStart': 9, 'duration': 3728, 'encodedBodySize': 1404, 'entryType': 'navigation', 'fetchStart': 0, 'initiatorType': 'navigation', 'loadEventEnd': 3728, 'loadEventStart': 3727, 'name': 'https://apps.sentinel-hub.com/eo-browser/?zoom=14&lat=57.54487&lng=35.39589&themeId=DEFAULT-THEME&visualizationUrl=U2FsdGVkX1%2FepgvG6E2WUC1LWyvzozgdjkA5aHcwtz4%2FebDwUxrBRALjOrL5NSiaoOk1%2FW87kt1n9TsdgnhdTam2mxn%2F%2FuoOrVZmT5TS38XrDAjkNPTPmc0J5eOcfLO4&datasetId=S2L2A&fromTime=2024-09-18T00%3A00%3A00.000Z&toTime=2024-09-18T23%3A59%3A59.999Z&layerId=1_TRUE_COLOR&demSource3D=%22MAPZEN%22', 'nextHopProtocol': 'h2', 'redirectCount': 0, 'redirectEnd': 0, 'redirectStart': 0, 'requestStart': 298, 'responseEnd': 400, 'responseStart': 400, 'responseStatus': 200, 'secureCon

# Проверка на незаполненность картинок

In [ ]:
# Можно зарегистрироваться на бесплатный 30 дневный чтобы работать через апишку и токены

In [167]:
import requests

response = requests.post('https://services.sentinel-hub.com/api/v1/process',
  headers={"Authorization" : "Bearer <your_access_token>"},
  json={
    "input": {
        "bounds": {
            "bbox": [
                13.822174072265625,
                45.85080395917834,
                14.55963134765625,
                46.29191774991382
            ]
        },
        "data": [{
            "type": "sentinel-2-l2a"
        }]
    },
    "evalscript": """
    //VERSION=3

    function setup() {
      return {
        input: ["B02", "B03", "B04"],
        output: {
          bands: 3
        }
      };
    }

    function evaluatePixel(
      sample,
      scenes,
      inputMetadata,
      customData,
      outputMetadata
    ) {
      return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
    }
    """
})

In [169]:
response

<Response [401]>